# Desafío Alura Store — Análisis y Recomendación


## Índice
- [0) Preparación](#0)
- [1) Importación de datos](#1)
- [2) Configuración de datos](#2)
- [3) Limpieza](#3)
- [4) Facturación por tienda](#4)
- [5) Categorías más vendidas](#5)
- [6) Calificación promedio por tienda](#6)
- [7) Productos más y menos vendidos](#7)
- [8) Costo de envío por tienda](#8)
- [9) Reporte y recomendación](#9)


<a id='0'></a>

## 0) Preparación

In [ ]:

import os, glob, math
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')


<a id='1'></a>

## 1) Importación de datos

In [ ]:

# from codigo_base import cargar_datos
# df = cargar_datos()


<a id='2'></a>

## 2) Configuración de datos

In [ ]:

DATA_DIR = 'data'
FILE_PATTERN = '*.csv'
SINGLE_CSV = None

COLUMN_ALIASES = {
    'store': ['store','tienda','seller_store','loja'],
    'order_id': ['order_id','id_pedido','pedido_id'],
    'product_id': ['product_id','id_producto','produto_id'],
    'product_name': ['product_name','nombre_producto','produto_nome','product'],
    'category': ['category','categoria','product_category','categoria_producto'],
    'quantity': ['quantity','cantidad','qty','qtd'],
    'price': ['price','precio','preco','unit_price'],
    'review_score': ['review_score','calificacion','nota','rating','review','score'],
    'shipping_cost': ['shipping_cost','costo_envio','frete','shipping_price','envio'],
    'customer_id': ['customer_id','id_cliente','cliente_id']
}

def _standardize_columns(df, aliases):
    cols = {c.lower().strip(): c for c in df.columns}
    result = {}
    for canonical, options in aliases.items():
        for opt in options:
            if opt in cols:
                result[cols[opt]] = canonical
                break
    return df.rename(columns=result)

def _read_many_csvs(data_dir, pattern):
    paths = sorted(glob.glob(os.path.join(data_dir, pattern)))
    if not paths: raise FileNotFoundError('No se encontraron CSV.')
    frames = []
    for p in paths:
        tmp = pd.read_csv(p, low_memory=False)
        tmp = _standardize_columns(tmp, COLUMN_ALIASES)
        if 'store' not in tmp.columns:
            tmp['store'] = os.path.splitext(os.path.basename(p))[0]
        frames.append(tmp)
    return pd.concat(frames, ignore_index=True)

def _read_single_csv(path):
    df = pd.read_csv(path, low_memory=False)
    df = _standardize_columns(df, COLUMN_ALIASES)
    if 'store' not in df.columns:
        raise ValueError('Falta columna store.')
    return df

if 'df' not in globals():
    df = _read_single_csv(SINGLE_CSV) if SINGLE_CSV else _read_many_csvs(DATA_DIR, FILE_PATTERN)

required = ['store','order_id','product_id','category','quantity','price','review_score','shipping_cost']
missing = [c for c in required if c not in df.columns]
if missing: raise ValueError(f'Faltan columnas: {missing}')

if 'product_name' not in df.columns:
    df['product_name'] = df['product_id'].astype(str)

for c in ['quantity','price','review_score','shipping_cost']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df['revenue'] = df['quantity'] * df['price']
df['units'] = df['quantity']
df.head(3)


<a id='3'></a>

## 3) Limpieza

In [ ]:

df_clean = df.dropna(subset=['store','order_id','product_id','category','quantity','price']).copy()
for c in ['quantity','price','shipping_cost','review_score']:
    if c in df_clean.columns:
        df_clean.loc[df_clean[c] < 0, c] = np.nan
stores = sorted(df_clean['store'].dropna().unique().tolist())
len(stores), stores[:10]


<a id='4'></a>

## 4) Facturación por tienda

In [ ]:

facturacion = (df_clean.groupby('store', as_index=False)
               .agg(revenue_total=('revenue','sum'),
                    unidades_total=('units','sum'))
               .sort_values('revenue_total', ascending=False))
display(facturacion)

plt.figure()
plt.bar(facturacion['store'], facturacion['revenue_total'])
plt.title('Facturación por tienda')
plt.xlabel('Tienda'); plt.ylabel('Facturación')
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

plt.figure()
plt.pie(facturacion['revenue_total'], labels=facturacion['store'], autopct='%1.1f%%')
plt.title('Participación de facturación'); plt.tight_layout(); plt.show()


<a id='5'></a>

## 5) Categorías más vendidas

In [ ]:

top_n = 5
cats = (df_clean.groupby(['store','category'], as_index=False)
        .agg(unidades=('units','sum'))
        .sort_values(['store','unidades'], ascending=[True,False]))
display(cats.head(10))
for s in stores:
    sub = cats[cats['store']==s].head(top_n)
    plt.figure()
    plt.bar(sub['category'].astype(str), sub['unidades'])
    plt.title(f'Top {top_n} categorías - {s}')
    plt.xlabel('Categoría'); plt.ylabel('Unidades')
    plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()


<a id='6'></a>

## 6) Calificación promedio por tienda

In [ ]:

if 'review_score' in df_clean.columns:
    ratings = (df_clean.dropna(subset=['review_score']).groupby('store', as_index=False)
               .agg(calificacion_promedio=('review_score','mean'),
                    cantidad_resenas=('review_score','count'))
               .sort_values('calificacion_promedio', ascending=False))
    display(ratings)
    plt.figure()
    plt.bar(ratings['store'], ratings['calificacion_promedio'])
    plt.title('Calificación promedio por tienda')
    plt.xlabel('Tienda'); plt.ylabel('Calificación')
    plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()
    plt.figure()
    df_clean['review_score'].dropna().plot(kind='hist', bins=10)
    plt.title('Distribución de reseñas'); plt.xlabel('Puntuación'); plt.ylabel('Frecuencia')
    plt.tight_layout(); plt.show()
else:
    print('review_score no disponible')


<a id='7'></a>

## 7) Productos más y menos vendidos

In [ ]:

prod = (df_clean.groupby(['store','product_id','product_name'], as_index=False)
        .agg(unidades=('units','sum'), revenue=('revenue','sum')))

def extremos_por_tienda(df_in, k=5):
    res = []
    for s in stores:
        sub = df_in[df_in['store']==s].sort_values('unidades', ascending=False)
        top = sub.head(k).assign(tipo='TOP')
        bottom = sub.sort_values('unidades', ascending=True).head(k).assign(tipo='BOTTOM')
        res.append(pd.concat([top,bottom], ignore_index=True))
    return pd.concat(res, ignore_index=True)

extremos = extremos_por_tienda(prod, k=5)
display(extremos)
for s in stores:
    sub = extremos[(extremos['store']==s) & (extremos['tipo']=='TOP')]
    plt.figure()
    plt.bar(sub['product_name'].astype(str), sub['unidades'])
    plt.title(f'TOP 5 productos - {s}')
    plt.xlabel('Producto'); plt.ylabel('Unidades')
    plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()


<a id='8'></a>

## 8) Costo de envío por tienda

In [ ]:

envio = (df_clean.dropna(subset=['shipping_cost']).groupby('store', as_index=False)
         .agg(costo_envio_promedio=('shipping_cost','mean'),
              costo_envio_mediano=('shipping_cost','median'),
              costo_envio_total=('shipping_cost','sum'))
         .sort_values('costo_envio_promedio', ascending=True))
display(envio)

plt.figure()
plt.bar(envio['store'], envio['costo_envio_promedio'])
plt.title('Costo promedio de envío')
plt.xlabel('Tienda'); plt.ylabel('Costo')
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

sample = df_clean.dropna(subset=['shipping_cost','revenue']).sample(min(len(df_clean), 5000), random_state=7)
plt.figure()
plt.scatter(sample['revenue'], sample['shipping_cost'], alpha=0.5)
plt.title('Costo de envío vs. valor del pedido')
plt.xlabel('Revenue'); plt.ylabel('Costo de envío')
plt.tight_layout(); plt.show()


<a id='9'></a>

## 9) Reporte y recomendación

In [ ]:

base = facturacion[['store','revenue_total','unidades_total']].copy()
if 'ratings' in locals():
    base = base.merge(ratings[['store','calificacion_promedio']], on='store', how='left')
else:
    base['calificacion_promedio'] = np.nan
base = base.merge(envio[['store','costo_envio_promedio']], on='store', how='left')

def minmax(s):
    s = s.astype(float)
    if s.nunique(dropna=True) <= 1: return pd.Series([0.5 if not math.isnan(x) else np.nan for x in s], index=s.index)
    return (s - s.min(skipna=True)) / (s.max(skipna=True) - s.min(skipna=True))

norm_rev = minmax(base['revenue_total'])
norm_units = minmax(base['unidades_total'])
norm_rate = minmax(base['calificacion_promedio'])
norm_ship = minmax(-base['costo_envio_promedio'])

weights = {'revenue':0.4,'units':0.2,'rating':0.25,'shipping':0.15}
base['score'] = (weights['revenue']*norm_rev.fillna(norm_rev.mean()) +
                 weights['units']*norm_units.fillna(norm_units.mean()) +
                 weights['rating']*norm_rate.fillna(norm_rate.mean()) +
                 weights['shipping']*norm_ship.fillna(norm_ship.mean()))
resultado = base.sort_values('score', ascending=False).reset_index(drop=True)
resultado['ranking'] = np.arange(1, len(resultado)+1)
display(resultado[['ranking','store','score','revenue_total','unidades_total','calificacion_promedio','costo_envio_promedio']])

peor_tienda = resultado.sort_values('score').iloc[0]['store']

reporte = []
reporte.append('# Informe ejecutivo - Alura Store')
reporte.append(f'_Generado: {datetime.now().strftime("%Y-%m-%d %H:%M")}_\n')
reporte.append('## Resumen')
reporte.append('1) Facturación por tienda')
reporte.append(facturacion.to_markdown(index=False))
reporte.append('\n2) Calificación promedio por tienda')
if 'ratings' in locals(): reporte.append(ratings.to_markdown(index=False))
else: reporte.append('_Sin reseñas válidas_')
reporte.append('\n3) Costo de envío por tienda')
reporte.append(envio.to_markdown(index=False))
reporte.append('\n## Recomendación')
reporte.append(f'Tienda sugerida para vender: **{peor_tienda}**.')
with open('Alura_Store_Reporte.md','w',encoding='utf-8') as f: f.write("\n".join(reporte))
'OK'
